In [25]:
from qiskit import *
from qiskit.algorithms.optimizers import SLSQP 
from qiskit.quantum_info import SparsePauliOp 
from qiskit.primitives import Estimator 
import numpy as np 
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from qiskit_nature import settings
settings.tensor_unwrapping = False
settings.use_pauli_sum_op = False

distance = 0.735
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 {}'.format(distance),
                     unit=DistanceUnit.ANGSTROM, charge=0, spin=0, basis='sto3g')

electronic_structure_problem = driver.run()

print('Electonic structue problem (Molecule) ')
print(electronic_structure_problem.molecule)
print('(Basis) = ', electronic_structure_problem.basis)
print('(num_spatial_orbitals) = ', electronic_structure_problem.num_spatial_orbitals)

Electonic structue problem (Molecule) 
Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		H	(0.0, 0.0, 0.0)
		H	(0.0, 0.0, 1.3889487015553204)
	Masses:
		H	1
		H	1
(Basis) =  ElectronicBasis.MO
(num_spatial_orbitals) =  2


In [26]:
mapper = JordanWignerMapper()
np_classical_solver = NumPyMinimumEigensolver()

/var/folders/n5/sr7wwb8n1wggd7jsk_k37l_w0000gn/T/ipykernel_13578/1989761901.py:2: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  np_classical_solver = NumPyMinimumEigensolver()


#### Calculate initial state to be provided to the QPE

In [28]:
initial_state=HartreeFock(
    electronic_structure_problem.num_spatial_orbitals,
    electronic_structure_problem.num_particles,
    mapper)

initial_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────

In [29]:
from qiskit.algorithms import HamiltonianPhaseEstimation
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver


FermionicOpWithCoefficients = electronic_structure_problem.hamiltonian.second_q_op()
print("Number of spin orbitals:", FermionicOpWithCoefficients.num_spin_orbitals, electronic_structure_problem.num_particles)

qubit_mapper = ParityMapper(num_particles=electronic_structure_problem.num_particles)
qubit_hamiltonian = mapper.map(FermionicOpWithCoefficients)


Number of spin orbitals: 4 (1, 1)


#### Classical solution

In [30]:
numpy_solver = NumPyMinimumEigensolver()
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
calc = GroundStateEigensolver(qubit_mapper, numpy_solver)

res = calc.solve(electronic_structure_problem)

print(res.eigenvalues.real)

[-1.85727503]


#### Quantum Phase estimation 

In [31]:
print('No of qubits required for the solution : ', qubit_hamiltonian.num_qubits)
sampler = Sampler()
pe = HamiltonianPhaseEstimation(num_evaluation_qubits=8, sampler=sampler)
result = pe.estimate(qubit_hamiltonian, initial_state)
print(result.most_likely_eigenvalue.real)

No of qubits required for the solution :  4
-1.85971577670655
